In [649]:
import pandas as pd
import numpy as np

df = pd.read_excel('Household income and expenditure.xlsx', index_col=False)


In [650]:
match = pd.read_csv('muni-reg.csv', sep=';', index_col=False)
match = match[['sourceCode', 'sourceName', 'targetCode', 'targetName']]
match['targetCode'] = match['targetCode'].str.replace("'", "", regex=False)
match['sourceCode'] = match['sourceCode'].str.replace("'", "", regex=False)
match['combined']=match.apply(lambda x:'%s %s' % (x['targetCode'],x['targetName']),axis=1)
match

,sourceCode,sourceName,targetCode,targetName,combined
0,020,Akaa,06,Pirkanmaa,06 Pirkanmaa
1,005,Alajärvi,14,South Ostrobothnia,14 South Ostrobothnia
2,009,Alavieska,17,North Ostrobothnia,17 North Ostrobothnia
3,010,Alavus,14,South Ostrobothnia,14 South Ostrobothnia
4,016,Asikkala,07,Päijät-Häme,07 Päijät-Häme
...,...,...,...,...,...
303,977,Ylivieska,17,North Ostrobothnia,17 North Ostrobothnia
304,980,Ylöjärvi,06,Pirkanmaa,06 Pirkanmaa
305,981,Ypäjä,05,Kanta-Häme,05 Kanta-Häme
306,989,Ähtäri,14,South Ostrobothnia,14 South Ostrobothnia


In [651]:
municipality_to_region = dict(zip(match['sourceName'], match['targetCode']))
muni_name_to_code = dict(zip(match['sourceName'], match['sourceCode']))

In [652]:
rows_per_region = 13

dfs = {}



for i in range(119):
    region = df.iloc[i*rows_per_region:(i+1)*rows_per_region]
    name = region['Unnamed: 0'].iloc[0]

    muni = ["Total"] * 23
    area = [name] * 23

    region = region.drop('Unnamed: 0', axis=1)
    region = region.T
    new_header = region.iloc[0]
    region = region[1:]
    region.columns = new_header
    region = region.reset_index()
    region = region.rename(columns={'index': 'year'})
    region['municipality'] = muni
    region['Area'] = area

    cols = list(region.columns.values)

    cols.insert(0, cols.pop(cols.index('municipality')))
    cols.insert(0, cols.pop(cols.index('Area')))
    region = region[cols]

    dfs[name] = region

dff = dfs['WHOLE COUNTRY']



dff



Unnamed: 1,Area,municipality,year,"B2N Operating surplus, net","B3N Mixed income, net","D11R Wages and salaries received, income","D12R Employer's social security contributions, income","D4OMR Property income, (incl. holding gains and losses), income","D4K Property income, expenditure",B5N Net national income,"D62R Social benefits other than social transfers in kind, income","D7R Other current transfers, income","D5K Income, property and other direct taxes excl. capital taxes, expenditure",D61K Employer's social security contributions,"D7K Other current transfers, expenditure","B6N Disposable income, net"
0,WHOLE COUNTRY,Total,2000,4293,5627,50915,12827,10877,1939,82600,22348,1281,19769,17583,2513,66364
1,WHOLE COUNTRY,Total,2001,4470,5869,53590,13768,9467,2145,85019,23008,1297,20258,18618,2718,67730
2,WHOLE COUNTRY,Total,2002,4970,5990,55468,13934,8809,1893,87278,24358,1297,20698,18607,2785,70843
3,WHOLE COUNTRY,Total,2003,5287,6069,57096,13896,9034,1676,89706,25372,1309,20553,18555,2814,74465
4,WHOLE COUNTRY,Total,2004,5530,6240,59468,14428,10239,1568,94337,26247,1265,21175,19200,2897,78577
5,WHOLE COUNTRY,Total,2005,5460,6496,62445,15205,10350,1826,98130,26975,1459,22165,20327,3055,81017
6,WHOLE COUNTRY,Total,2006,5631,6843,65609,15924,11916,2367,103556,27605,1505,22912,21732,3171,84851
7,WHOLE COUNTRY,Total,2007,5790,7985,69286,16762,15304,3698,111429,28150,1522,24332,22864,3222,90683
8,WHOLE COUNTRY,Total,2008,6332,7736,73949,17772,14469,5003,115255,29515,1672,25583,23857,3371,93631
9,WHOLE COUNTRY,Total,2009,6536,7522,73260,17355,9865,2185,112353,32394,1618,24034,23497,3431,95403


In [653]:
combined_df = pd.concat(dfs.values(), ignore_index=True)

In [654]:
final_exp = combined_df[combined_df['Area'].str.startswith('MK', na=False)]
final_exp['Area'] = final_exp['Area'].str.replace('MK', '', regex=False).str.split(' ').str[0]

C:\Users\DEll\AppData\Local\Temp\ipykernel_2892\3695514812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_exp['Area'] = final_exp['Area'].str.replace('MK', '', regex=False).str.split(' ').str[0]


In [655]:
final_exp = final_exp.rename(columns={'Area': 'region'})
final_exp.index.name = None
final_exp

Unnamed: 1,region,municipality,year,"B2N Operating surplus, net","B3N Mixed income, net","D11R Wages and salaries received, income","D12R Employer's social security contributions, income","D4OMR Property income, (incl. holding gains and losses), income","D4K Property income, expenditure",B5N Net national income,"D62R Social benefits other than social transfers in kind, income","D7R Other current transfers, income","D5K Income, property and other direct taxes excl. capital taxes, expenditure",D61K Employer's social security contributions,"D7K Other current transfers, expenditure","B6N Disposable income, net"
138,01,Total,2000,1201,1080,18799,4676,4500,534,29720,5664,311,7232,6341,676,21447
139,01,Total,2001,1256,1320,19686,5035,3355,608,30045,5879,325,7428,6746,734,21341
140,01,Total,2002,1416,1377,20344,5100,3043,548,30730,6271,323,7514,6760,745,22305
141,01,Total,2003,1452,1400,20723,5046,3352,466,31508,6583,322,7427,6687,745,23552
142,01,Total,2004,1438,1576,21559,5234,3785,436,33156,6904,315,7692,6918,766,24999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,21,Total,2018,57,45,537,110,100,5,844,218,13,155,169,34,717
617,21,Total,2019,52,59,561,113,104,5,884,226,14,171,176,36,741
618,21,Total,2020,55,60,543,103,106,4,863,240,13,170,166,37,744
619,21,Total,2021,53,62,564,114,181,6,968,242,14,189,180,42,812


In [656]:
id_cols = ['year', 'municipality', 'Area']

# Define the columns to melt (all the economic indicators)
value_cols = [
    'B2N Operating surplus, net',
    'B3N Mixed income, net', 
    'D11R Wages and salaries received, income',
    "D12R Employer's social security contributions, income",
    'D4OMR  Property income, (incl. holding gains and losses), income',
    'D4K Property income, expenditure',
    'B5N Net national income',
    'D62R Social benefits other than social transfers in kind, income',
    'D7R Other current transfers, income',
    'D5K Income, property and other direct taxes excl. capital taxes, expenditure',
    "D61K Employer's social security contributions",
    'D7K Other current transfers, expenditure',
    'B6N Disposable income, net'
]

for i in dfs: 
    dfs[i] = dfs[i].melt(
        id_vars=id_cols,
        value_vars=value_cols,
        var_name='factor',
        value_name='value'
    )

    dfs[i] = dfs[i][['Area', 'municipality', 'year', 'factor', 'value']]





In [657]:
df2 = pd.read_excel('Debt 03.xlsx', index_col=False)

df2 = df2.replace('...', np.nan)
df2 = df2.replace('.', np.nan)

df2 = df2.rename(columns={'Unnamed: 0': 'municipality', 'Unnamed: 1': 'debt type'})

muni = df2['municipality'][1:]
debt = df2['debt type'][1:]




In [658]:
rows_per_region = 5

df2s = []
col = df2.iloc[0][2:19]

chunk_size = 17
num_chunks = 22
chunks = []
total_cols = len(df2.columns)


for i in range(num_chunks):
    start_idx = i * chunk_size + 2
    end_idx = min((i + 1) * chunk_size + 2, total_cols) 
    chunk = df2.iloc[:, start_idx:end_idx]
    year = [chunk.columns[0]]*1545
    chunk = chunk.iloc[1:]
    chunk.columns = col
    chunk['year'] = year
    chunk['municipality'] = muni
    chunk['debt'] = debt
    chunk = chunk.rename(columns={'50th percentile of debt (median)': 'Median of debt'})
    chunk = chunk[['municipality', 'year', 'Number of indebted household-dwelling units', 'debt','Median of debt', 'Mean of debt']]
    chunk = chunk.ffill()
    chunks.append(chunk)

# Access individual chunks
test = chunks[21] # First 10 columns


C:\Users\DEll\AppData\Local\Temp\ipykernel_2892\4241189103.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk = chunk.ffill()
C:\Users\DEll\AppData\Local\Temp\ipykernel_2892\4241189103.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk = chunk.ffill()
C:\Users\DEll\AppData\Local\Temp\ipykernel_2892\4241189103.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('

In [659]:
test.ffill()

,municipality,year,Number of indebted household-dwelling units,debt,Median of debt,Mean of debt
1,WHOLE COUNTRY,2024,1453864,All debts,46595,94401
2,WHOLE COUNTRY,2024,809411,Housing loan debts,89829,115962
3,WHOLE COUNTRY,2024,1111101,Consumption debts and other debts,15593,26791
4,WHOLE COUNTRY,2024,123478,Debts charged on business activities and a sou...,61166,110287
5,WHOLE COUNTRY,2024,123478,Study loan debts,61166,110287
...,...,...,...,...,...,...
1541,Äänekoski,2024,4121,All debts,40416,66848
1542,Äänekoski,2024,2591,Housing loan debts,53352,67739
1543,Äänekoski,2024,3086,Consumption debts and other debts,13877,23154
1544,Äänekoski,2024,333,Debts charged on business activities and a sou...,36356,85628


In [660]:
for i in range(len(chunks)):
    df_pivoted = chunks[i].pivot(index=['municipality', 'year', 'Number of indebted household-dwelling units'], 
                        columns='debt', 
                        values=['Median of debt', 'Mean of debt'])

    # Flatten column names
    df_pivoted.columns = [f"{col[0]} for {col[1]}" for col in df_pivoted.columns]

    # Reset index to make municipality and year regular columns
    df_pivoted = df_pivoted.reset_index()
    df_pivoted['region'] = df_pivoted['municipality'].map(municipality_to_region)
    df_pivoted['municipality'] = df_pivoted['municipality'].map(muni_name_to_code)
    chunks[i] = df_pivoted

In [661]:
final_debt = pd.concat(chunks, ignore_index=True)
cols = list(final_debt.columns.values)
cols.insert(0, cols.pop(cols.index('region')))
final_debt = final_debt[cols]
final_debt

,region,municipality,year,Number of indebted household-dwelling units,Median of debt for All debts,Median of debt for Consumption debts and other debts,Median of debt for Debts charged on business activities and a source of income,Median of debt for Housing loan debts,Median of debt for Study loan debts,Mean of debt for All debts,Mean of debt for Consumption debts and other debts,Mean of debt for Debts charged on business activities and a source of income,Mean of debt for Housing loan debts,Mean of debt for Study loan debts
0,06,020,2003,337,NaN,NaN,33072.0,NaN,NaN,NaN,NaN,63081.0,NaN,NaN
1,06,020,2003,651,NaN,NaN,NaN,NaN,4629.0,NaN,NaN,NaN,NaN,6004.0
2,06,020,2003,2459,NaN,NaN,NaN,53668.0,NaN,NaN,NaN,NaN,65372.0,NaN
3,06,020,2003,2890,NaN,9749.0,NaN,NaN,NaN,NaN,16805.0,NaN,NaN,NaN
4,06,020,2003,4023,36390.0,NaN,NaN,NaN,NaN,58285.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30837,14,989,2024,1105,38972.0,NaN,NaN,NaN,NaN,71129.0,NaN,NaN,NaN,NaN
30838,13,992,2024,333,NaN,NaN,36356.0,NaN,36356.0,NaN,NaN,85628.0,NaN,85628.0
30839,13,992,2024,2591,NaN,NaN,NaN,53352.0,NaN,NaN,NaN,NaN,67739.0,NaN
30840,13,992,2024,3086,NaN,13877.0,NaN,NaN,NaN,NaN,23154.0,NaN,NaN,NaN


In [662]:
final_debt

,region,municipality,year,Number of indebted household-dwelling units,Median of debt for All debts,Median of debt for Consumption debts and other debts,Median of debt for Debts charged on business activities and a source of income,Median of debt for Housing loan debts,Median of debt for Study loan debts,Mean of debt for All debts,Mean of debt for Consumption debts and other debts,Mean of debt for Debts charged on business activities and a source of income,Mean of debt for Housing loan debts,Mean of debt for Study loan debts
0,06,020,2003,337,NaN,NaN,33072.0,NaN,NaN,NaN,NaN,63081.0,NaN,NaN
1,06,020,2003,651,NaN,NaN,NaN,NaN,4629.0,NaN,NaN,NaN,NaN,6004.0
2,06,020,2003,2459,NaN,NaN,NaN,53668.0,NaN,NaN,NaN,NaN,65372.0,NaN
3,06,020,2003,2890,NaN,9749.0,NaN,NaN,NaN,NaN,16805.0,NaN,NaN,NaN
4,06,020,2003,4023,36390.0,NaN,NaN,NaN,NaN,58285.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30837,14,989,2024,1105,38972.0,NaN,NaN,NaN,NaN,71129.0,NaN,NaN,NaN,NaN
30838,13,992,2024,333,NaN,NaN,36356.0,NaN,36356.0,NaN,NaN,85628.0,NaN,85628.0
30839,13,992,2024,2591,NaN,NaN,NaN,53352.0,NaN,NaN,NaN,NaN,67739.0,NaN
30840,13,992,2024,3086,NaN,13877.0,NaN,NaN,NaN,NaN,23154.0,NaN,NaN,NaN


In [663]:
combined_df = pd.concat([final_debt, final_exp], 
                               ignore_index=True, 
                               sort=False)
combined_df
dummy_combined = combined_df.copy()

In [664]:
def med(med_type, combined_df):

    def calculate_regional_weighted_median(group):
        """
        Calculate population-weighted median for a region using municipality medians
        and number of indebted households as weights
        """
        medians = group[med_type].values  # or whatever your median column is named
        weights = group['Number of indebted household-dwelling units'].values
        
        # Remove any NaN values
        mask = ~(np.isnan(medians) | np.isnan(weights)) & (weights > 0)
        medians = medians[mask]
        weights = weights[mask]
        
        if len(medians) == 0:
            return np.nan
        
        # Sort by median values
        sorted_indices = np.argsort(medians)
        sorted_medians = medians[sorted_indices]
        sorted_weights = weights[sorted_indices]
        
        # Calculate cumulative weights
        cumsum_weights = np.cumsum(sorted_weights)
        total_weight = cumsum_weights[-1]
        
        # Find the weighted median position
        median_position = total_weight / 2
        
        # Find where cumulative weight crosses the median position
        median_idx = np.searchsorted(cumsum_weights, median_position, side='right')
        
        # Handle edge cases
        if median_idx == 0:
            return sorted_medians[0]
        elif median_idx >= len(sorted_medians):
            return sorted_medians[-1]
        else:
            # For exact median, you might want to interpolate between values
            # but typically we just take the value at the median position
            return sorted_medians[median_idx]


    regional_medians = final_debt.groupby(['region', 'year']).apply(calculate_regional_weighted_median).reset_index()
    regional_medians.columns = ['region', 'year', 'Median debt']

    regional_medians['municipality'] = ['Total']*418
    regional_medians = regional_medians[['region', 'municipality', 'year', 'Median debt']]

    combined_df = combined_df.merge(regional_medians, on=['region', 'municipality', 'year'], how='left')

    mask = (combined_df['municipality'] == 'Total') & (combined_df[med_type].isna())
    combined_df.loc[mask, med_type] = combined_df.loc[mask, 'Median debt']
    combined_df = combined_df.drop('Median debt', axis=1)
    return combined_df

In [665]:
for i in combined_df.columns.values:
    if i.startswith('Median'):
        combined_df = med(i, dummy_combined)

C:\Users\DEll\AppData\Local\Temp\ipykernel_2892\2042207449.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regional_medians = final_debt.groupby(['region', 'year']).apply(calculate_regional_weighted_median).reset_index()
C:\Users\DEll\AppData\Local\Temp\ipykernel_2892\2042207449.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regional_medians = final_debt.groupby(['region', 'year']).apply(calculat

In [666]:
combined_df

,region,municipality,year,Number of indebted household-dwelling units,Median of debt for All debts,Median of debt for Consumption debts and other debts,Median of debt for Debts charged on business activities and a source of income,Median of debt for Housing loan debts,Median of debt for Study loan debts,Mean of debt for All debts,...,"D12R Employer's social security contributions, income","D4OMR Property income, (incl. holding gains and losses), income","D4K Property income, expenditure",B5N Net national income,"D62R Social benefits other than social transfers in kind, income","D7R Other current transfers, income","D5K Income, property and other direct taxes excl. capital taxes, expenditure",D61K Employer's social security contributions,"D7K Other current transfers, expenditure","B6N Disposable income, net"
0,06,020,2003,337.0,NaN,NaN,33072.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06,020,2003,651.0,NaN,NaN,NaN,NaN,4629.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06,020,2003,2459.0,NaN,NaN,NaN,53668.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06,020,2003,2890.0,NaN,9749.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06,020,2003,4023.0,36390.0,NaN,NaN,NaN,NaN,58285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31274,21,Total,2018,NaN,NaN,NaN,NaN,NaN,53626.0,NaN,...,110,100,5,844,218,13,155,169,34,717
31275,21,Total,2019,NaN,NaN,NaN,NaN,NaN,55975.0,NaN,...,113,104,5,884,226,14,171,176,36,741
31276,21,Total,2020,NaN,NaN,NaN,NaN,NaN,65660.0,NaN,...,103,106,4,863,240,13,170,166,37,744
31277,21,Total,2021,NaN,NaN,NaN,NaN,NaN,64339.0,NaN,...,114,181,6,968,242,14,189,180,42,812


In [667]:
for i in combined_df.columns.values:
    if i.startswith('Mean'):
        regional_avg_debt = combined_df.groupby(['region', 'year'])[i].mean()
        mask = (combined_df['municipality'] == 'Total') & (combined_df[i].isna())
        combined_df.loc[mask, i] = combined_df.loc[mask, ['region', 'year']].apply(
            lambda row: regional_avg_debt.get((row['region'], row['year']), None), 
            axis=1)

In [668]:
# Remove column D
combined_df = combined_df.drop(combined_df.columns[3], axis=1)  # Removes column D (index 3)

# Define a function to get first non-null value
def first_valid(series):
    return series.dropna().iloc[0] if len(series.dropna()) > 0 else None

# Group and aggregate
combined_df = combined_df.groupby(['region', 'municipality', 'year']).agg(first_valid).reset_index()
combined_df

,region,municipality,year,Median of debt for All debts,Median of debt for Consumption debts and other debts,Median of debt for Debts charged on business activities and a source of income,Median of debt for Housing loan debts,Median of debt for Study loan debts,Mean of debt for All debts,Mean of debt for Consumption debts and other debts,...,"D12R Employer's social security contributions, income","D4OMR Property income, (incl. holding gains and losses), income","D4K Property income, expenditure",B5N Net national income,"D62R Social benefits other than social transfers in kind, income","D7R Other current transfers, income","D5K Income, property and other direct taxes excl. capital taxes, expenditure",D61K Employer's social security contributions,"D7K Other current transfers, expenditure","B6N Disposable income, net"
0,01,018,2003,59388.0,11716.0,40324.0,61190.0,4344.0,80012.0000,20022.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,018,2004,58848.0,12650.0,39612.0,65961.0,4353.0,83471.0000,21301.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,018,2005,69631.0,14614.0,43714.0,79899.0,4612.0,105231.0000,31828.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,018,2006,81678.0,16863.0,39098.0,88128.0,4851.0,110952.0000,31722.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,018,2007,84166.0,16356.0,41275.0,97776.0,3894.0,118083.0000,31443.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7208,21,Total,2018,NaN,NaN,NaN,NaN,53626.0,135757.5000,54686.4375,...,110.0,100.0,5.0,844.0,218.0,13.0,155.0,169.0,34.0,717.0
7209,21,Total,2019,NaN,NaN,NaN,NaN,55975.0,136165.5000,54918.5000,...,113.0,104.0,5.0,884.0,226.0,14.0,171.0,176.0,36.0,741.0
7210,21,Total,2020,NaN,NaN,NaN,NaN,65660.0,139930.0625,56271.7500,...,103.0,106.0,4.0,863.0,240.0,13.0,170.0,166.0,37.0,744.0
7211,21,Total,2021,NaN,NaN,NaN,NaN,64339.0,136181.1875,55449.0625,...,114.0,181.0,6.0,968.0,242.0,14.0,189.0,180.0,42.0,812.0


In [670]:
combined_df.to_csv('Household income, expenditure and debt.csv', index=False)

In [ ]:
chunks[21].iloc[:5]#['Share of indebted household-dwelling units (%)']

ex = {}

for i in range(len(chunks)):
    ex[str(2003+i)] = chunks[i].iloc[:5]['Share of indebted household-dwelling units (%)']

share = pd.DataFrame.from_dict(ex)
share['debt type'] = debt

share

KeyError: 'Share of indebted household-dwelling units (%)'